# GeneriCorp Employee Handbook
This notebook demonstrates how I simulated the documents representing GeneriCorp's employee handbook. I chose NOT to use ChatGPT directly since it would have been too tedious to generate all these by hand! As such, I'll demonstrate for you how we can use the OpenAI API to generate these documents in an automated, efficient manner.

## Notebook Setup

In [1]:
# Importing the necessary Python libraries
import os
import json
import pandas as pd
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [2]:
# Instantiating the chat model
chat_model = ChatOpenAI(api_key = os.environ['OPENAI_API_KEY'],
                        model = 'gpt-4o-mini')

In [3]:
# Loading the company history from file
with open('../../genericorp/about-gc/company-history.md', 'r', encoding = 'utf-8') as file:
    company_history = file.read()

## Generating Handbook Topics

In [4]:
# Creating the topic generation prompt
TOPIC_GENERATION_PROMPT = '''Imagine a fictional company called GeneriCorp. Their company history is as follows:

{company_history}

Please generate a list of topics they may be found within the employee handbook. Please output them in a Python comma-separated list format. Do not provide any additional text aside from the Python list.'''

# Instantiating the output parser
output_parser = CommaSeparatedListOutputParser()

# Setting up the chat prompt template
topic_generation_prompt_template = PromptTemplate(
    template = TOPIC_GENERATION_PROMPT + '\n{format_instructions}',
    input_variables = ['company_history'],
    partial_variables = {'format_instructions': output_parser.get_format_instructions()}
)

# Creating the topic generation chain
topic_generation_chain = topic_generation_prompt_template | chat_model | output_parser

In [ ]:
# Setting the name of a topics cache file
TOPICS_CACHE_FILE = 'handbook_topics.json'

# Checking if cached topics exist
if os.path.exists(TOPICS_CACHE_FILE):
    
    # Loading topics from cache
    with open(TOPICS_CACHE_FILE, 'r') as f:
        handbook_topics = json.load(f)
        
else:

    # Generating topics using the AI model
    handbook_topics = topic_generation_chain.invoke({"company_history": company_history})
    
    # Caching the results
    with open(TOPICS_CACHE_FILE, 'w') as f:
        json.dump(handbook_topics, f)

# Replacing spaces with underscores in handbook topics
handbook_topics = [topic.replace(' ', '_') for topic in handbook_topics]

print(f"Number of topics: {len(handbook_topics)}")
print("Topics:", handbook_topics)

Number of topics: 21
Topics: ['company_culture', 'code_of_conduct', 'benefits_and_compensation', 'employee_wellness_programs', 'diversity_and_inclusion', 'harassment_and_discrimination_policies', 'workplace_safety', 'remote_work_policies', 'performance_reviews', 'training_and_development', 'confidentiality_agreements', 'intellectual_property', 'employee_resources', 'leave_policies', 'travel_policies', 'communication_guidelines', 'ethics_and_compliance', 'work-life_balance', 'grievance_procedures', 'team_building_activities', 'recognition_programs']


## Generating the Handbook Pages from the Topics

In [ ]:
# Creating the handbook page generation prompt
HANDBOOK_PAGE_GENERATION_PROMPT = '''Imagine a fictional company called GeneriCorp. Their company history is as follows:
{company_history}
'''